In [ ]:
import pandas as pd
df = pd.read_csv("housing.xls")
#df.head()

In [ ]:
numeric_df = df.select_dtypes(include=["int64", "float64"])
for col in numeric_df.columns:
    print(f"=== Statistics for {col} ===")
    print(f"Count: {numeric_df[col].count()}")
    print(f"Mean: {numeric_df[col].mean()}")
    print(f"Median: {numeric_df[col].median()}")
    print(f"Standard Deviation: {numeric_df[col].std()}")
    print(f"Minimum: {numeric_df[col].min()}")
    print(f"Maximum: {numeric_df[col].max()}")
    print(f"25th Percentile (Q1): {numeric_df[col].quantile(0.25)}")
    print(f"50th Percentile (Median): {numeric_df[col].quantile(0.50)}")
    print(f"75th Percentile (Q3): {numeric_df[col].quantile(0.75)}")
    print(f"10th Percentile: {numeric_df[col].quantile(0.10)}")
    print(f"90th Percentile: {numeric_df[col].quantile(0.90)}")
    print(f"Interquartile Range (IQR): {numeric_df[col].quantile(0.75) - numeric_df[col].quantile(0.25)}")
    print("\n")

In [ ]:
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)
#percentage of missing values
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values per column:\n", missing_percentage)


In [ ]:
# Imputation for numerical variables
# Mean imputation for price and area
df['price'].fillna(df['price'].mean(), inplace=True)
df['area'].fillna(df['area'].mean(), inplace=True)

# Median imputation for bedrooms and bathrooms (robust to outliers)
df['bedrooms'].fillna(df['bedrooms'].median(), inplace=True)
df['bathrooms'].fillna(df['bathrooms'].median(), inplace=True)

# Mode imputation for stories and parking (categorical/discrete)
df['stories'].fillna(df['stories'].mode()[0], inplace=True)
df['parking'].fillna(df['parking'].mode()[0], inplace=True)

# Verify no missing values remain
print("\nMissing values after imputation:\n", df.isnull().sum())


In [ ]:
# Check for duplicate rows
duplicate_rows = df.duplicated()
print("Number of duplicate rows:", duplicate_rows.sum())
#Display the duplicate rows
duplicates = df[duplicate_rows]
duplicates

In [18]:
# Remove duplicate rows
df_cleaned = df.drop_duplicates()
# Verify duplicates are removed
print("Number of rows after removing duplicates:", df_cleaned.shape[0])

Number of rows after removing duplicates: 545


In [19]:
df.dtypes

price                 int64
area                float64
bedrooms              int64
bathrooms           float64
stories               int64
mainroad             object
guestroom            object
basement             object
hotwaterheating      object
airconditioning      object
parking               int64
prefarea             object
furnishingstatus     object
dtype: object

In [21]:
# Describe to spot strange ranges
df.describe()
# Check manually suspicious columns
print("Bedrooms - unique values:", df['bedrooms'].unique())
print("Stories - unique values:", df['stories'].unique())
print("Parking - unique values:", df['parking'].unique())

Bedrooms - unique values: [  4   3   5 400   2   6   1]
Stories - unique values: [3 4 2 1]
Parking - unique values: [2 3 0 1]


In [22]:
# Check unique categorical values
for col in df.select_dtypes(include=['object']).columns:
    print(col, df[col].unique())

mainroad ['yes' 'no']
guestroom ['no' 'yes']
basement ['no' 'yes']
hotwaterheating ['no' 'yes']
airconditioning ['yes' 'no']
prefarea ['yes' 'no']
furnishingstatus ['furnished' 'semi-furnished' 'unfurnished']


In [23]:
df['area'].sample(10)

278    3400.0
53     5150.0
275    4032.0
47     6600.0
301    3520.0
451    6750.0
92     4800.0
528    3970.0
32     4880.0
379    3520.0
Name: area, dtype: float64

In [38]:
# Replace unrealistic bedroom values (>= 20 considered unrealistic)
median_bedrooms = df['bedrooms'].median()
df.loc[df['bedrooms'] >= 20, 'bedrooms'] = median_bedrooms
print(df['bedrooms'].max())

6


In [40]:
#Convert all Yes/No values to lowercase, clean spaces
categorical_cols = df.select_dtypes(include='object').columns

for col in categorical_cols:
    df[col] = df[col].str.lower().str.strip()

In [41]:
#Fix known categorical patterns
df['furnishingstatus'] = df['furnishingstatus'].replace({
    'semi-furnished': 'semi furnished',
    'semi furnished ': 'semi furnished',
    'furnished ': 'furnished'
})

In [42]:
# Ensure numerical columns are numeric
#Sometimes CSV files load numbers as strings:
num_cols = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [43]:
df.loc[df['parking'] < 0, 'parking'] = 0
df.loc[df['parking'] > 4, 'parking'] = df['parking'].mode()[0]

In [45]:
import numpy as np
# Select ONLY numerical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df_num = df[num_cols]
# ---- IQR METHOD ----
outlier_summary = {}
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    outlier_summary[col] = len(outliers)

# Show IQR-based outliers
print("=== IQR Outliers Count ===")
for col, count in outlier_summary.items():
    print(f"{col}: {count}")
# ---- Z-SCORE METHOD ----
from scipy import stats
zscore_summary = {}
for col in num_cols:
    z_scores = np.abs(stats.zscore(df[col].dropna()))
    z_outliers = len(df[col].dropna()[z_scores > 3])
    zscore_summary[col] = z_outliers
print("=== Z-score Outliers Count (|z| > 3) ===")
for col, count in zscore_summary.items():
    print(f"{col}: {count}")

=== IQR Outliers Count ===
price: 15
area: 11
bedrooms: 12
bathrooms: 1
stories: 41
parking: 12
=== Z-score Outliers Count (|z| > 3) ===
price: 6
area: 6
bedrooms: 2
bathrooms: 11
stories: 0
parking: 0


In [46]:
from scipy.stats.mstats import winsorize

#1. Winsorize price (keep domain shape but reduce influence of extreme values)
df['price'] = winsorize(df['price'], limits=[0.01, 0.01])

#2. Remove unrealistic bedroom values (e.g., > 20)
df = df[df['bedrooms'] <= 20]

#3. No changes for area, bathrooms, stories, parking
# All their outliers are domain-valid
print("Outlier handling completed successfully.")


Outlier handling completed successfully.


In [47]:
from sklearn.preprocessing import StandardScaler

# Create scaler
scaler = StandardScaler()

# Columns to scale
columns_to_scale = ["price", "area"]

# Fit and transform
df_scaled = df.copy()
df_scaled[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Show sample of scaled data
df_scaled.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,3.353076,1.070539,4,2.0,3,yes,no,no,no,yes,2,yes,furnished
1,3.353076,1.789576,4,4.0,4,yes,no,no,no,yes,3,no,furnished
2,3.353076,2.256483,3,2.0,2,yes,no,yes,no,no,2,yes,semi furnished
3,3.353076,NaN,4,2.0,2,yes,no,yes,no,yes,3,yes,furnished
4,3.353076,NaN,4,1.0,2,yes,yes,yes,no,yes,2,no,furnished


In [50]:
#before label encoding
df.select_dtypes(include=['object']).head()

,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus
0,yes,no,no,no,yes,yes,furnished
1,yes,no,no,no,yes,no,furnished
2,yes,no,yes,no,no,yes,semi furnished
3,yes,no,yes,no,yes,yes,furnished
4,yes,yes,yes,no,yes,no,furnished


In [51]:
#after label encoding
from sklearn.preprocessing import LabelEncoder
binary_cols = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
label_encoders = {}
df_label = df.copy()
for col in binary_cols:
    le = LabelEncoder()
    df_label[col] = le.fit_transform(df_label[col])
    label_encoders[col] = le
df_label[binary_cols].head()

,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea
0,1,0,0,0,1,1
1,1,0,0,0,1,0
2,1,0,1,0,0,1
3,1,0,1,0,1,1
4,1,1,1,0,1,0


In [ ]:
#before encoding
df['furnishingstatus'].head()

In [ ]:
#after encoding
df_onehot = pd.get_dummies(df, columns=["furnishingstatus"], drop_first=False)
df_onehot.head()